# El Fin (Final Exam) for AMPTH-2017/APMA E-207

**Harvard University**<br>
**Fall 2018**<br>
**Instructors: Rahul Dave**<br>
**Due Date:** Monday, December 17th, 2018 at 11:59pm

**Instructions:**

- Upload your final answers in the form of a Jupyter notebook containing all work to Canvas.

- Structure your notebook and your work to maximize readability.

### Collaborators

#### Group 44 - Braavos

**Joe Davison<br>
Anna Davydova<br>
Michael S. Emanuel<br>
Dylan Randle<br>**

In [ ]:
import numpy as np
import scipy.stats
from scipy.stats import logistic
import pandas as pd
import pymc3 as pm
import theano.tensor as tt
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn.apionly as sns

In [ ]:
# Additional imports
import pickle
from IPython.display import display
import warnings
from typing import Dict

In [ ]:
# Utilities for serializing variables
def load_vartbl(fname: str) -> Dict:
    """Load a dictionary of variables from a pickled file"""
    try:
        with open(fname, 'rb') as fh:
            vartbl = pickle.load(fh)
    except:
        vartbl = dict()
    return vartbl


def save_vartbl(vartbl: Dict, fname: str) -> None:
    """Save a dictionary of variables to the given file with pickle"""
    with open(fname, 'wb') as fh:
        pickle.dump(vartbl, fh)

In [ ]:
# Turn off deprecation warning (too noisy)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=mpl.MatplotlibDeprecationWarning)

## Q1: GLMs with correlation

### The dataset: A Bangladesh Contraception use census

This problem is based on one-two (12H1 and continuations ) from your textbook. The data is in the file `bangladesh.csv`. These data are from the 1988 Bangladesh Fertility Survey. Each row is one of 1934 women. There are six variables:

- (1) `district`: ID number of administrative district each woman resided in
- (2) `use.contraception`: An indicator (0/1) of whether the woman was using contraception
- (3) `urban`: An indicator (0/1) of whether the woman lived in a city, as opposed to living in a rural area
- (4) `woman`: a number indexing a single woman in this survey
- (5) `living.chidren`: the number of children living with a woman
- (6) `age.centered`: a continuous variable representing the age of the woman with the sample mean subtracted

We need to make sure that the cluster variable, district, is a contiguous set of integers, so that we can use the index to differentiate the districts easily while sampling ((look at the Chimpanzee models we did in lab to understand the indexing). So create a new contiguous integer index to represent the districts. Give it a new column in the dataframe, such as `district.id`.

You will be investigating the dependence of contraception use on the district in which the survey was done. Specifically, we will want to regularize estimates from those districts where very few women were surveyed. We will further want to investigate whether the areas of residence (urban or rural) within a district impacts a woman's use of contraception.

Feel free to indulge in any exploratory visualization which helps you understand the dataset better.

In [ ]:
# Serialize variables for GLM quesion
fname: str = 'glm.pickle'
vartbl: Dict = load_vartbl(fname)

In [ ]:
# Load the dataset
df = pd.read_csv('bangladesh.csv', sep=';')
# Generate map of distinct districts
district_id_map = {d:i for i, d in enumerate(sorted(set(df.district)))}
# Add new column district_id to the dataset; these are contiguous integers 0-59
df['district_id'] = df.district.map(district_id_map)

# Rename the columns to avoid the confusing '.' in column names
df.rename(axis='columns', inplace=True, mapper=
    {'use.contraception':'use_contraception',
     'living.children':'living_children',
     'age.centered':'age_centered',
     })

# The number of districts
num_districts: int = len(district_id_map)

# Create a dataset aggregated by district (sufficient statistic for the by district model)
agg_tbl = {
        'woman': ['count'],
        'use_contraception': ['sum']
        } 
df_district = df.groupby(by=df.district_id).agg(agg_tbl)
df_district.columns = ["_".join(x) for x in df_district.columns.ravel()]
# The number of women sampled in each district
district_count = df_district.woman_count

# Set the number of samples for this problem (used in multiple parts)
num_samples: int = 11000
num_tune: int = 1000

In [ ]:
# Review the dataset
display(df.sample(25).sort_values(by=['woman']))

## Part A

We will use `use.contraception` as a Bernoulli response variable. 

When we say "fit" below, we mean, specify the model, plot its graph, sample from it, do some tests, and forest-plot and summarize the posteriors, at the very least.

**A1** Fit a traditional "fixed-effects" model which sets up district-specific intercepts, each with its own Normal(0, 10) prior. That is, the intercept is modeled something like 

```python
alpha_district = pm.Normal('alpha_district', 0, 10, shape=num_districts)
p=pm.math.invlogit(alpha_district[df.district_id])
```

Why should there not be any overall intercept in this model? 

**A2** Fit a multi-level "varying-effects" model with an overall intercept `alpha`, and district-specific intercepts `alpha_district`. Assume that the overall intercept has a Normal(0, 10) prior, while the district specific intercepts are all drawn from the **same** normal distribution with mean 0 and standard deviation $\sigma$. Let $\sigma$ be drawn from HalfCauchy(2). The setup of this model is similar to the per-chimanzee models in the prosocial chimanzee labs.

**A3** What does a posterior-predictive sample in this model look like? What is the difference between district specific posterior predictives and woman specific posterior predictives. In other words, how might you model the posterior predictive for a new woman being from a particular district vs that os a new woman in the entire sample? This is a word answer; no programming required.

**A4** Plot the predicted proportions of women in each district using contraception against the id of the district, in both models. How do these models disagree? Look at the extreme values of predicted contraceptive use in the fixed effects model. How is the disagreement in these cases?

**A5** Plot the absolute value of the difference in probability of contraceptive use against the number of women sampled in each district. What do you see?

**A1** Fit a traditional "fixed-effects" model which sets up district-specific intercepts, each with its own Normal(0, 10) prior. That is, the intercept is modeled something like

alpha_district = pm.Normal('alpha_district', 0, 10, shape=num_districts)
p=pm.math.invlogit(alpha_district[df.district_id])

**View Data Aggregeated by District**

In [ ]:
fig, ax = plt.subplots(figsize=[12,8])
ax.set_title('Histogram of Count of Women in Each District')
ax.set_xlabel('Count of Women')
ax.set_ylabel('Frequency')
ax.hist(df_district.woman_count, bins=20)
ax.grid()

In [ ]:
fig, ax = plt.subplots(figsize=[12,8])
ax.set_title('Count of Women in Each District')
ax.set_xlabel('District ID')
ax.set_ylabel('Count of Women')
ax.bar(df_district.index.values, df_district.woman_count)
ax.grid()

We can see that there are three districts with small sample sizes.  These will be hard to estimate in this model.

**Specify the Fixed Effects Model and Draw Samples**

In [ ]:
# Define the fixed-effects model
with pm.Model() as model_fe:
    # Set the prior for the intercept in each district
    alpha_district = pm.Normal(name='alpha_district', mu=0.0, sd=10.0, shape=num_districts)
    # Set the probability that each woman uses contraception in this model
    # It depends only on the district she lives in
    p = pm.math.invlogit(alpha_district[df.district_id])
    # The response variable - whether this woman used contraception; modeled as Bernoulli
    # Bind this to the observed values
    use_contraception = pm.Bernoulli('use_contraception', p=p, observed=df['use_contraception'])

# Sample from the fixed-effects model
try:
    trace_fe = vartbl['trace_fe']
    print(f'Loaded samples for the Fixed Effects model in trace_fe.')
except:
    with model_fe:
        trace_fe = pm.sample(draws=num_samples, tune=num_tune, chains=2, cores=1)
    vartbl['trace_fe'] = trace_fe
    save_vartbl(vartbl, fname)

**Tabular Summary of the 60 alpha Parameters for Fixed Effects Model**

In [ ]:
summary_fe = pm.summary(trace_fe)
display(summary_fe)

**Plot Graphs for Fixed Effects Model: Traceplot, Autocorrplot**

In [ ]:
fig = pm.traceplot(trace_fe)

In [ ]:
fig = pm.autocorrplot(trace_fe)

With 60 variables (one for each district) it can be a bit difficult to visualize what's going on.  The autocorr plots look OK with very low correlations after the first handful of steps.  Most of the alpha parameters for each district are clustered together.  A few of them have very wide bases; these are the districts with small number of samples.

**Whisker Plot of Posterior Intercepts for Each District**

In [ ]:
# Samples of alpha as a an Nx60 array
alpha_samples_fe = trace_fe.get_values('alpha_district')
# Arrange the alpha samples into a dataframe for plotting
col_names_fe = [f'alpha_{i}' for i in range(num_districts)]
df_alpha_samples_fe = pd.DataFrame(data=alpha_samples_fe, columns = col_names_fe)

# Generate a whisker plot
mpl.rcParams.update({'font.size': 16})
fig, ax = plt.subplots(figsize=[12,24])
ax.set_title('Fixed Effects Model: Whisker Plot for alpha by District')
ax.set_xlabel('Alpha')
ax.set_ylabel('District ID')
ax.set_xlim(-5, 5)
ax.set_xticks(np.arange(-5,6))
ax.grid()
sns.boxplot(data=alpha_samples_fe, orient='h', ax=ax)

Unlike the earlier plots, this one is very readable.  It shows the alpha parameter (intercept) for each district.  A high alpha means women in that district are more likely to use contraception.

**Forest Plot for Fixed Effects Model**

In [ ]:
fig = plt.figure(figsize=(12,20))
gs = pm.forestplot(trace_fe)
gs.figure = fig
ax1, ax2 = fig.axes
ax1 = ax1.set_xlim(0.8, 1.2)

**Tests for Fixed Effects Model**

In [ ]:
# Check n_eff, the number of effective draws
n_eff = summary_fe['n_eff']
n_eff_mean = np.mean(n_eff)
n_eff_min = np.min(n_eff)
print(f'Mean of n_eff for {num_districts} alpha parameters is {n_eff_mean:0.0f}')
print(f'Min of n_eff is {n_eff_min:0.0f}')

# Check Rhat, the correlation between of parameter estimates between chains
rhat = summary_fe['Rhat']
rhat_mean = np.mean(rhat)
rhat_min = np.min(rhat)
print(f'\nMean of RHat for {num_districts} alpha parameters is {rhat_mean:0.3f}')
print(f'Min of Rhat is {rhat_min:0.3f}')

The number of effective draws is high and the correlations between chains are good.  These results seem fine, and don't indicate any sampling problems.  Also there were no warnings from the samplers.

**Plot Posterior for Fixed Effects Model: alpha for Each District**

In [ ]:
fig = pm.plot_posterior(trace_fe)

**Summarize the Posteriors for Fixed Effects Model**

We can start summarizing the posteriors with the table displayed well above with summary statistics for each parameter.  It's a lot to take in because there are 60 districts.  Let's do some higher level summaries below.

In [ ]:
# Compute the weighted average of the mean and sd of alpha 
# weighting is by the number of women in each district
alpha_mean_wtd = np.average(a=summary_fe['mean'], weights=district_count)
alpha_sd_wtd = np.average(a=summary_fe['sd'], weights=district_count)

print(f'Weighted average of mean and sd alpha, weighted by number of women in each district:')
print(f'alpha_mean_wtd: {alpha_mean_wtd:0.4f}')
print(f'alpha_sd_wtd:    {alpha_sd_wtd:0.4f}')

This is telling us that overall, women in this survey are more likely not to use contraception (mean alpha weighted by size is negative).  The size of the negative alpha is somewhat larger than the weighted average standard deviation, so the effect has a reasonable size.  This lines up with the overall mean rate of contraception use in the surve, 39.25%.

**Why should there not be any overall intercept in this model?**

This model fits a parameter $\alpha$ to each district the determines the probability $p$ that women in the district use contraception according to a logistic distribution.  Adding an overall intercept would just create a new parameter that was colinear with the 60 alphas, one for each district.  It would be redundant.  It could create sampling problems due to misindetification, since a model where the overall intercept was 0.1 higher and all the $alpha_i$ were 0.1 lower would be the same.  It's worth pointing out here that if the priors on the individual districts were grouped more tightly around zero, while the prior for the overall rate was wider, then this model specification might have some benefits.

**A2** Fit a multi-level "varying-effects" model with an overall intercept `alpha`, and district-specific intercepts `alpha_district`. Assume that the overall intercept has a Normal(0, 10) prior, while the district specific intercepts are all drawn from the **same** normal distribution with mean 0 and standard deviation $\sigma$. Let $\sigma$ be drawn from HalfCauchy(2). The setup of this model is similar to the per-chimanzee models in the prosocial chimanzee labs.

**Specify the Variable Effects Model and Draw Samples**

In [ ]:
# Define the varying-effects model
with pm.Model() as model_ve:
    # Set the prior for the overall intercept
    alpha = pm.Normal(name='alpha', mu=0.0, sd=10.0)
    # Set the width sigma for the variability among districts
    sigma = pm.HalfCauchy(name='sigma', beta=2.0)
    # Set the district-specific alphas to have mean 0 and standard deviation sigma
    alpha_district = pm.Normal(name='alpha_district', mu=0.0, sd=sigma, shape=num_districts)    
    # Set the probability that each woman uses contraception in this model
    # It depends only on the district she lives in
    p = pm.math.invlogit(alpha + alpha_district[df.district_id])
    # The response variable - whether this woman used contraception; modeled as Bernoulli
    # Bind this to the observed values
    use_contraception = pm.Bernoulli('use_contraception', p=p, observed=df['use_contraception'])

# Sample from the variable-effects model
try:
    trace_ve = vartbl['trace_ve']
    print(f'Loaded samples for the Variable Effects model in trace_ve.')
except:
    with model_ve:
        trace_ve = pm.sample(draws=num_samples, tune=num_tune, chains=2, cores=1)
    vartbl['trace_ve'] = trace_ve
    save_vartbl(vartbl, fname)

**Tabular Summary of the 60 alpha Parameters for Variable Effects Model**

In [ ]:
summary_ve = pm.summary(trace_ve)
display(summary_ve)

**Plot Graphs for Fixed Effects Model: Traceplot, Autocorrplot**

In [ ]:
fig = pm.traceplot(trace_ve)

In [ ]:
fig = pm.autocorrplot(trace_fe)

The additional structure of the variable effects model makes it easier to visualize.  We can see that the alpha and sigma parameters have reasonable looking distributions that are consistent on the two chains.  The traceplots look like pure white noise.  We can see more of a pattern in the individual districts, where the center of each district reflects overall contraception usage in the district and the width reflects the number of women polled.

**Whisker Plot of Posterior Intercepts for Each District in Variable Effects Model**

In [ ]:
# Samples of alpha as an Nx60 array
alpha_overall_ve = trace_ve.get_values('alpha')
sigma_ve = trace_ve.get_values('sigma')
alpha_district_samples_ve = trace_ve.get_values('alpha_district')
alpha_samples_ve = np.hstack([alpha_overall_ve.reshape(-1,1),
                              sigma_ve.reshape(-1,1),
                              alpha_district_samples_ve])
y_labels = ['intercept', 'sigma'] + [f'district {i}' for i in range(num_districts)]

# Arrange the alpha samples into a dataframe for plotting
col_names_ve = ['alpha', 'sigma'] + [f'alpha_{i}' for i in range(num_districts)]
df_alpha_samples_ve = pd.DataFrame(data=alpha_samples_ve, columns = col_names_ve)

# Generate a whisker plot
mpl.rcParams.update({'font.size': 16})
fig, ax = plt.subplots(figsize=[12,24])
ax.set_title('Variable Effects Model: Whisker Plot for alpha by District')
ax.set_xlabel('Parameter Value')
ax.set_ylabel('Parameter Name')
ax.set_xlim(-5, 5)
ax.set_xticks(np.arange(-5,6))
ax.grid()
sns.boxplot(data=alpha_samples_ve, orient='h', ax=ax)
ax.set_yticklabels(y_labels)
display(ax)

One thing that jumps out here is that the pooled alpha (overall intercept of contraceptive usage) is quite tight compared to most of the districts.  That's good! It shows that the information pooling is working and will help estimate the districts with small sample sizes.  The estimate on $\sigma$, the standard deviation of the global intercept $\alpha$, is also quite tight.

**Forest Plot for Variable Effects Model**

In [ ]:
fig = plt.figure(figsize=(12,20))
gs = pm.forestplot(trace_ve)
gs.figure = fig
ax1, ax2 = fig.axes
ax1.set_xlim(0.8, 1.2)

**Tests for Variable Effects Model**

In [ ]:
# Check n_eff, the number of effective draws
n_eff = summary_ve['n_eff']
n_eff_alpha = n_eff[0]
n_eff_sigma = n_eff[-1]
n_eff_alphas_mean = np.mean(n_eff[1:-1])
n_eff_alphas_min = np.min(n_eff[1:-1])
print(f'n_eff for overall intercept alpha: {n_eff_alpha:0.0f}')
print(f'n_eff for sigma (stdev of alpha):  {n_eff_sigma:0.0f}')
print(f'Mean of n_eff for {num_districts} alpha_district parameters is {n_eff_mean:0.0f}')
print(f'Min of n_eff is {n_eff_alphas_min:0.0f}')

# Check Rhat, the correlation between of parameter estimates between chains
rhat = summary_ve['Rhat']
rhat_alpha = rhat[0]
rhat_sigma = rhat[-1]
rhat_alphas_mean = np.mean(rhat[1:-1])
rhat_alphas_min = np.min(rhat[1:-1])
print(f'\nRHat for overall intercept alpha: {rhat_alpha:0.3f}')
print(f'RHat for sigma (stdev ov alpha):  {rhat_sigma:0.3f}')
print(f'Mean of RHat for {num_districts} alpha_district parameters is {rhat_mean:0.3f}')
print(f'Min of Rhat is {rhat_alphas_min:0.3f}')

The number of effective parameters for $\alpha$ and $\sigma$, describing the overall mean, are a bit smaller than for the district level parameters.  But they are still quite high and completely adequate.  The minimum for the districts is also OK.
The RHat parameters are all very close to 1.0.  These tests are all consistent with good sampling behavior.

**Plot Posterior for Variable Effects Model: alpha for Each District**

In [ ]:
fig = pm.plot_posterior(trace_ve)

**Summarize the Posteriors for Variable Effects Model**

In [ ]:
# Compute the weighted average of the mean and sd of alpha 
# weighting is by the number of women in each district
alpha_mean = summary_ve['mean']['alpha']
alpha_std = summary_ve['sd']['alpha']
sigma_mean = summary_ve['mean']['sigma']
sigma_std = summary_ve['sd']['sigma']

print(f'alpha (overall intercept)')
print(f'mean: {alpha_mean:0.3f}')
print(f'std : {sigma_std:0.3f}')
print()
print(f'sigma (variability of districts)')
print(f'mean: {sigma_mean:0.3f}')
print(f'std : {sigma_std:0.3f}')

alphas_mean_wtd = np.average(a=summary_ve['mean'][1:-1], weights=district_count)
alphas_sd_wtd = np.average(a=summary_ve['sd'][1:-1], weights=district_count)

print(f'\nWeighted average of mean and std alpha by district, weighted by number of women in each district:')
print(f'mean: {alphas_mean_wtd:0.4f}')
print(f'std:  {alphas_sd_wtd:0.4f}')

The new model is easier to interpret.  The global intercept $\alpha$ has a mean of -0.54 and standard deviation of -0.085. It reflects the overall prevalence of contraceptive use across districts.  

$\sigma$ has a mean of 0.523 and a standard deviation of 0.085.  It reflects how much individual district vary from the global intercept $\alpha$.

The estimates on both $\alpha$ and $\sigma$ are fairly tight, reflecting that the model has successfully pooled information across the 60 districts.

The mean of $\alpha_i$ over the districts is 0.073.  It is close to zero because most of the overall departure from 50/50 contraceptive use is being absorbed in the global intercept $\alpha$.

**A3** What does a posterior-predictive sample in this model look like? What is the difference between district specific posterior predictives and woman specific posterior predictives. In other words, how might you model the posterior predictive for a new woman being from a particular district vs that of a new woman in the entire sample? This is a word answer; no programming required.

One posterior sample in this model assigns a global intercept $\alpha$, a global variability among districts $\sigma$, and draws $\alpha_i$ for the 60 districts.  (This is the posterior over parameter space, not the posterior-predictive over y-space).  The probability $p_i$ that women in a given district use contraceptive is the inverse logistic function of $\alpha + \alpha_i$.

We can then generate a posterior predictive sample by specifying the number of women in each of the 60 districts.  The probability that one woman who lives in district $i$ uses contraception is $p_i$ as described above.  The probability that $k_i$ of the $n_i$ women living in district $i$ will use contraception will be a binomial distribution with parameters $n_i$, $k_i$ and $p_i$, since the sum of Bernoulli trials is distributed binomially.

On the other hand, suppose we don't know anything about a woman and we want to predict the probability she uses contraception.  We can follow a simple ancestral sampling strategy.  First we model the probability that she lives in each of the districts, which we can do based on the overall population breakdown.  Then we take a weighted average over the probabilities for women living in each district.

**A4** Plot the predicted proportions of women in each district using contraception against the id of the district, in both models. How do these models disagree? Look at the extreme values of predicted contraceptive use in the fixed effects model. How is the disagreement in these cases?

In [ ]:
# Generate posterior predictive samples in both models
num_samples_ppc: int = 10000
try:
    post_pred_fe = vartbl['post_pred_fe']
    post_pred_ve = vartbl['post_pred_ve']
    print(f'Loaded posterior predictive samples for Fixed Effects and Varying Effects models.')
except:
    post_pred_fe = pm.sample_ppc(trace_fe, samples=num_samples_ppc, model=model_fe)
    post_pred_ve = pm.sample_ppc(trace_ve, samples=num_samples_ppc, model=model_ve)
    vartbl['post_pred_fe'] = post_pred_fe
    vartbl['post_pred_ve'] = post_pred_ve
    save_vartbl(vartbl, fname)

# Compute the mean contraception use in posterior samples of each model
df['use_contraception_fe'] = np.mean(post_pred_fe['use_contraception'], axis=0)
df['use_contraception_ve'] = np.mean(post_pred_ve['use_contraception'], axis=0)

# Update the aggregated contraception use in each district
agg_tbl = {
        'woman': ['count'],
        'use_contraception': ['mean'],
        'use_contraception_fe': ['mean'],
        'use_contraception_ve': ['mean'],        
        } 
df_district = df.groupby(by=df.district_id).agg(agg_tbl)
df_district.columns = ["_".join(x) for x in df_district.columns.ravel()]
# Change column names to make model suffix at the end of the name
df_district.rename(axis='columns', inplace=True, mapper=
    {'use_contraception_fe_mean':'use_contraception_mean_fe',
     'use_contraception_ve_mean':'use_contraception_mean_ve',
     })

In [ ]:
display(df_district)

In [ ]:
# Set up horizontal bar chart
district_id_agg = df_district.index.values
# Spacing between models in each district
space = 0.3
plot_y1 = district_id_agg - space
plot_y2 = district_id_agg
plot_y3 = district_id_agg + space
# Height of each horizontal bar
height = 0.2

# Plot contraception use for each district
fig, ax = plt.subplots(figsize=[16,20])
ax.set_title('Contraception Use vs. DistrictID by Model')
ax.set_xlabel('Contraception Use in District')
ax.set_ylabel('District ID')
ax.set_ylim(0, 60)
ax.barh(y=plot_y1, width=df_district.use_contraception_mean, height=height, label='Data', color='r')
ax.barh(y=plot_y2, width=df_district.use_contraception_mean_fe, height=height, label='FE', color='g')
ax.barh(y=plot_y3, width=df_district.use_contraception_mean_ve, height=height, label='VE', color='b')
ax.legend()
ax.grid()

**How do these models disagree?<br>
Look at the extreme values of predicted contraceptive use in the fixed effects model. 
How is the disagreement in these cases?**

In [ ]:
# Get districts with min and max contraception use in the FE model
district_min_fe = np.argmin(df_district.use_contraception_mean_fe)
district_max_fe = np.argmax(df_district.use_contraception_mean_fe)
# Display results
print(f'District ID {district_min_fe} has Minimum Contraception in Fixed Effect Model')
display(df_district.loc[district_min_fe])
print(f'District ID {district_max_fe} has Maximum Contraception in Fixed Effect Model')
display(df_district.loc[district_max_fe])

With 60 districts, it can hard to read off the district numbers, so we can cross reference the chart to to the table above.  It's easiest to spot the districts where contraception use is high.  District ID 2 has 100\% contraceptive use in the data, but based on a tiny sample of 2 responedents.  The fixed effects model has a much weaker prior and predicts that women in this district will use contraceptives 95.7\% of the time.  The varying effects model benefits from information pooling across all the districts and does far more shrinkage to the population mean for a district with such a small sample.  It predicts 45.7\% contraceptive use in this district.

District ID 10 has the lowest predicted contraception use among all 60 districts in the fixed effect model.  The FE model predicts 0.5\% contraception use, whereas the varying effect model predicts 18.2\% contraception use.  This case is more interesting than the last one.  With 21 women in the sample count, and 0 of them reporting contraception use, it appears at first glance that in this case, the FE model is more accurate, and the VE model may be overcorrecting for the population mean.

In general, these two extreme cases give us a good idea for how the two models disagree.  The FE model comes quite close to simply predicting that each district has a contraception use rate close to the observed rate in the data.  This makes it vulnerable to making extreme predictions in districts that have a small number of responses.  The VE model is pooling across all the districts, and performing shrikage to the population mean.  It does more shrinkage in districts with a small response rate.  The two models with vary the most in districts with a small response count that is far from the population mean.

**A5** Plot the absolute value of the difference in probability of contraceptive use against the number of women sampled in each district. What do you see?

In [ ]:
# Assemble series for this plot
# the x-axis is the number of women in the district
plot_x = df_district.woman_count.values
# the y-axis is the absolute value of the difference between the FE and VE models
plot_y = np.abs(df_district.use_contraception_mean_fe - df_district.use_contraception_mean_ve).values

# Generate the plot
fig, ax = plt.subplots(figsize=[12,8])
ax.set_title('Difference Between FE and VE Models vs. Sample Size')
ax.set_xlabel('Sample Size (# Women Polled)')
ax.set_ylabel('Abs(pred_FE - pred_VE)')
ax.plot(plot_x, plot_y, color='b', marker='o', markersize=8, linewidth=0)
ax.grid()

As expected, the smaller the sample size, the larger the difference between the FE and VE models.  This is because the VE model is performing more shrinkage on districts with a smaller sample size.  We could have gotten a stronger regression fit if we added a second explanatory variable, the absolute value of the difference between the sample contraception use rate in each district and the overall population contraception use rate.

## Part B.

Let us now fit a model with both varying intercepts by `district_id` (like we did in the varying effects model above) and varying slopes of `urban` by `district_id`. To do this, we will 

- (a) have an overall intercept, call it `alpha`
- (b) have an overall slope of `urban`, call it `beta`.
- (c) have district specific intercepts `alpha_district`
- (d) district specific slopes for `urban`, `beta_district`
- (e) model the co-relation between these slopes and intercepts. 

We have not modelled covariance and correlation before, so look at http://am207.info/wiki/corr.html for notes on how this is done.

To see the ideas behind this, see section 13.2.2 on the income data  from your textbook (included as a pdf in this zip). Feel free to use [code with attribution from Osvaldo Martin](https://github.com/aloctavodia/Statistical-Rethinking-with-Python-and-PyMC3/blob/master/Chp_13.ipynb)..with attribution and understanding...there is some sweet pymc3 technical wrangling in there.

**B1** Write down the model as a pymc3 specification and look at its graph. Note that this model builds a 60 by 2 matrix with `alpha_district` values in the first column and `beta_district` values in the second. By assumption, the first column and the second column have correlation structure given by an LKJ prior, but there is no explicit correlation among the rows. In other words, the correlation matrix is 2x2 (not 60x60). Make sure to obtain the value of the off-diagonal correlation as a `pm.Deterministic`. (See Osvaldo Martin's code above)

**B2**: Sample from the posterior of the model above *with a target acceptance rate of .9 or more*. (Sampling takes me 7 minutes 30 seconds on my 2013 Macbook Air). Comment on the quality of the samples obtained.

**B3** Propose a method based on the reparametrization trick for multi-variate gaussians) of improving the quality of the samples obtained and implement it. (A hint can be obtained from here: https://docs.pymc.io/api/distributions/multivariate.html#pymc3.distributions.multivariate.MvNormal . Using that hint lowered the sampling time to 2.5 minutes on my laptop).

**B4** Inspect the trace of the correlation between the intercepts and slopes, plotting the correlation marginal. What does this correlation tell you about the pattern of contraceptive use in the sample? It might help to plot the mean (or median) varying effect estimates for both the intercepts and slopes, by district. Then you can visualize the correlation and maybe more easily think through what it means to have a particular correlation. Also plot the predicted proportion of women using contraception, with urban women on one axis and rural on the other.  Finally, also plot the difference between urban and rural probabilities against rural probabilities. All of these will help you interpret your findings. (Hint: think in terms of low or high rural contraceptive use)

**B5** Add additional "slope" terms (one-by-one) into the model for 

- (a) the centered-age of the women and 
- (b) an indicator for whether the women have a small number or large number of existing kids in the house (you can treat 1-2 kids as low, 3-4 as high, but you might want to experiment with this split). 

Are any of these effects significant? Are any significant effects similar over the urban/rural divide?

**B6** Use WAIC to compare your models. What are your conclusions?

**B1** Write down the model as a pymc3 specification and look at its graph. Note that this model builds a 60 by 2 matrix with `alpha_district` values in the first column and `beta_district` values in the second. By assumption, the first column and the second column have correlation structure given by an LKJ prior, but there is no explicit correlation among the rows. In other words, the correlation matrix is 2x2 (not 60x60). Make sure to obtain the value of the off-diagonal correlation as a `pm.Deterministic`. (See Osvaldo Martin's code above)

In [ ]:
def pm_make_cov(sigma_priors, corr_coeffs, ndim):
    """Assemble a covariance matrix single variable standard deviations and correlation coefficients"""
    # Citation: AM 207 lecture notes: http://am207.info/wiki/corr.html
    # Diagonal matrix of standard deviation for each varialbes
    sigma_matrix = tt.nlinalg.diag(sigma_priors)
    # A symmetric nxn matrix has n choose 2 = n(n-1)/2 distinct elements
    n_elem = int(ndim * (ndim - 1) / 2)
    # Convert between array indexing and [i, j) indexing
    tri_index = np.zeros([ndim, ndim], dtype=int)
    tri_index[np.triu_indices(ndim, k=1)] = np.arange(n_elem)
    tri_index[np.triu_indices(ndim, k=1)[::-1]] = np.arange(n_elem)
    # Assemble the covariance matrix using the equation
    # CovMat = DiagMat * CorrMat * DiagMat
    corr_matrix = corr_coeffs[tri_index]
    corr_matrix = tt.fill_diagonal(corr_matrix, 1)
    return tt.nlinalg.matrix_dot(sigma_matrix, corr_matrix, sigma_matrix)

In [ ]:
# Define a varying slopes model incorporating a beta_urban term
with pm.Model() as model_vs:  
    # Set the prior for the overall intercept
    alpha = pm.Normal(name='alpha', mu=0.0, sd=10.0)
    # Set the prior for the overall intercept on urban, beta
    beta = pm.Normal(name='beta', mu=0.0, sd=10.0)
    
    # Citation: http://am207.info/wiki/corr.html for code controlling correlation structure
    # The parameter nu is the prior on correlation; 0 is uniform, infinity is no corelation
    nu = pm.Uniform('nu', 1.0, 5.0)
    # The number of dimensions here is 2: correlation structure is bewteen alpha and beta by district
    num_factors: int = 2
    # Sample the correlation coefficients using the LKJ distribution
    corr_coeffs = pm.LKJCorr('corr_coeffs', nu, num_factors)

    # Sample the variances of the single factors
    sigma_priors = tt.stack([pm.Lognormal('sigma_prior_alpha', mu=0.0, tau=1.0),
                             pm.Lognormal('sigma_prior_beta', mu=0.0, tau=1.0)])

    # Make the covariance matrix as a Theano tensor
    cov = pm.Deterministic('cov', pm_make_cov(sigma_priors, corr_coeffs, num_factors))
    # The multivariate Gaussian of (alpha, beta) by district
    theta_district = pm.MvNormal('theta_district', mu=[0.0, 0.0], cov=cov, shape=(num_districts, num_factors))   

    # The vector of standard deviations for each variable; size num_factors x num_factors
    # Citation: efficient generation of sigmas and rhos from cov
    # https://github.com/aloctavodia/Statistical-Rethinking-with-Python-and-PyMC3/blob/master/Chp_13.ipynb
    sigmas = pm.Deterministic('sigmas', tt.sqrt(tt.diag(cov)))
    # correlation matrix (num_factors x num_factors)
    rhos = pm.Deterministic('rhos', tt.diag(sigmas**-1).dot(cov.dot(tt.diag(sigmas**-1))))

    # Extract the standard deviations of alpha and beta, and the correlation coefficient rho
    sigma_alpha = pm.Deterministic('sigma_alpha', sigmas[0])
    sigma_beta = pm.Deterministic('sigma_beta', sigmas[1])
    rho = pm.Deterministic('rho', rhos[0, 0])

    # Extract alpha_district and beta_district from theta_district
    alpha_district = pm.Deterministic('alpha_district', theta_district[:,0])
    beta_district = pm.Deterministic('beta_district', theta_district[:, 1])

    # Set the probability that each woman uses contraception in this model
    # It depends on the district she lives in and whether the district is urban
    # p = pm.math.invlogit(alpha + alpha_district[df.district_id] + 
    #                      (beta + beta_district[df.district_id]) * df.urban)
    p = pm.math.invlogit(alpha + theta_district[df.district_id, 0] + 
                         (beta + theta_district[df.district_id, 1]) * df.urban)

    # The response variable - whether this woman used contraception; modeled as Bernoulli
    # Bind this to the observed values
    use_contraception = pm.Bernoulli('use_contraception', p=p, observed=df['use_contraception'])

In [ ]:
# Generate graphs for each model
graph_fe = pm.model_to_graphviz(model_fe)
graph_ve = pm.model_to_graphviz(model_ve)
graph_vs = pm.model_to_graphviz(model_vs)

In [ ]:
# The Fixed Effect model
print('Fixed Effect Model Graph')
display(graph_fe)

In [ ]:
# The Variable Effect model
print('Variable Effect Model Graph')
display(graph_ve)

In [ ]:
# The Variable Slope model
print('Variable Slope Model Graph')
display(graph_vs)

**B2**: Sample from the posterior of the model above *with a target acceptance rate of .9 or more*. (Sampling takes me 7 minutes 30 seconds on my 2013 Macbook Air). Comment on the quality of the samples obtained.

In [ ]:
# Sample from the varying-slope model
try:
    trace_vs = vartbl['trace_vs']
    print(f'Loaded samples for the Variable Slopes model in trace_vs.')
except:
    with model_vs:
        nuts_kwargs = {'target_accept': 0.90}
        trace_vs = pm.sample(draws=num_samples, tune=num_tune, nuts_kwargs=nuts_kwargs, chains=2, cores=1)
    vartbl['trace_vs'] = trace_vs
    save_vartbl(vartbl, fname)

# Summary of the varying-slope model
summary_vs = pm.summary(trace_vs)
# List of parameters to report
params_all = summary_vs.index.values
num_pairs = num_factors * (num_factors-1) // 2
# The "key" parameters
params_key = \
    ['alpha', 'beta', 'nu', 'sigma_prior_alpha', 'sigma_prior_beta'] + \
    [f'sigmas__{i}' for i in range(num_factors)] + \
    [f'corr_coeffs__{i}' for i in range(num_pairs)] + \
    [f'cov__{i}_{j}' for i in range(num_factors) for j in range(num_factors)]
# The parameters pertaining to different districts
params_district = \
        [f'alpha_district__{i}' for i in range(num_districts)] + \
        [f'beta_district__{i}' for i in range(num_districts)]
# The parameters to display
params = params_key + params_district
display(summary_vs.loc[params])

**Comment on the quality of the samples obtained above**

In [ ]:
# Variable name for the traceplot
varnames = ['alpha', 'beta', 'nu', 'sigma_prior_alpha', 'sigma_prior_beta', 'sigmas', 'rho', 'cov']
# Generate traceplot
fig = pm.traceplot(trace_vs, varnames=varnames)

In [ ]:
# Generate autocorr plot
fig = pm.autocorrplot(trace_vs, varnames=varnames[0:6])

The quality of these samples is perhaps adequate, but not great.  The first warning sign came as, well, a warning from the sampler.  Each chain warned of 1 divergence during sampling.  (And this was with a 90% acceptance rate).  It also warned that the number of effective parameters was below 25% for some parameters.

The table above puts the parameters into a more "strategic" order, with the long vectors of alpha and beta by district at the bottom so they don't overwhelm the first few.  We can focus on these parameters for the plots as well, because the district level parameters seem OK.

The traceplots above don't present any obvious problems.  The autocorr plots are more informative.  We can see that there is quite a bit of correlation for the priors on the sigma of alpha and beta.  The covariance entries couldn't be plotted because pymc3 complained that the the array was too deep.

The number of effective parameters for sigma_prior_alpha, sigma_prior_beta, the sigmas, the correlation coefficients, and the covariances are all too small relative to the number of samples drawn.  Indeed this deficiency triggered a runtime warning from the sampler, complaining that n_eff was below 25% for some of the samples.

**B3** Propose a method based on the reparametrization trick for multi-variate gaussians) of improving the quality of the samples obtained and implement it. (A hint can be obtained from here: https://docs.pymc.io/api/distributions/multivariate.html#pymc3.distributions.multivariate.MvNormal . Using that hint lowered the sampling time to 2.5 minutes on my laptop).

There are two key ideas and they are both included in the hint cited above.  The first idea is that instead of sampling correlation coefficients and standard deviations directly, it is more efficient to sample the entries of a Cholesky factor matrix.  The second idea, which is more important for sampling performance, is the core of the reparameterization trick.  In one dimension, the reparameterization trick tells us that if we want to draw samples for a variable x with mean mu and standard deviation sigma, it is often faster and numerically easier on our sampling back end to instead draw samples for a variable z that is distributed as the standard normal with mean = 0 and standard deviation = 1.  Then we can construct the samples for x by applying the derministic rule 
$$x_i = \sigma \cdot z_i + mu$$

The reparameterization trick for multivariate Guassians is completely analogous.  The one difference is that we need to use the "matrix square root" of the covariance matrix, which is its Cholesky factor L.  So the analogous equation in the multidimensional case is
$$x_i = L \cdot z_i \cdot L^T + \mu$$

This is exactly the strategy carried out in the reparameterized model below.  The strategy subtantially sped up sampling.  On a Windows platform unfortunately parallel processing doesn't work.  Runtime on the first sampler for 2 chains with 12,000 samples (2,000 tuning, 10,000 retained) was 10:06 minutes per chain or 22 minutes total.  With the improved version, sampling one chain took about 3:41, a speed-up of 2.74 times.  Exact hardware configuration is a custom built PC with an AMD Ryzen Threadripper 2990WX 32 Core processor running at 3.00 GHz.  Sadly having 32 cores isn't much help when pymc3 on Windows is stuck running on one core at a time...  This notebook is set up to load the samples from a persisted data file if possible, so samples are not run redundantly.

In [ ]:
# Define a varying slopes model incorporating a beta_urban term
with pm.Model() as model_vsr:
    # Citation: ideas to efficiently reparameterize samples from a MV Gaussian
    # https://docs.pymc.io/api/distributions/multivariate.html#pymc3.distributions.multivariate.MvNormal
    # Set the prior for the overall intercept
    alpha = pm.Normal(name='alpha', mu=0.0, sd=10.0)
    # Set the prior for the overall intercept on urban, beta
    beta = pm.Normal(name='beta', mu=0.0, sd=10.0)
    
    # Sample the variances of the single factors
    # sd_dist = pm.HalfCauchy.dist(beta=2.5, shape=num_factors)
    sd_dist = pm.Lognormal.dist(mu=0.0, tau=1.0, shape=num_factors)
    # The parameter nu is the prior on correlation; 0 is uniform, infinity is no corelation
    eta = pm.Uniform('nu', 1.0, 5.0)
    # The number of dimensions here is 2: correlation structure is bewteen alpha and beta by district
    num_factors: int = 2
    # Sample the correlation coefficients using the LKJ distribution
    chol_packed = pm.LKJCholeskyCov('chol_packed', n=num_factors, eta=eta, sd_dist = sd_dist)
    # Expand the packed Cholesky matrix to full size
    chol = pm.Deterministic('chol', pm.expand_packed_triangular(num_factors, chol_packed))
    # Make the covariance matrix by multiplying out the cholesky factor by its transpose
    cov = pm.Deterministic('cov', tt.dot(chol, chol.T))
    # The multivariate Gaussian of (alpha, beta) by district
    # Decompose this into a "raw" part and then scale it
    theta_raw = pm.Normal(name='theta_raw', mu=0.0, sd=1.0, shape=(num_districts, num_factors))   
    # Now scale these to have the desired covariance structure
    theta_district = pm.Deterministic(name='theta_district', var=tt.dot(chol, theta_raw.T).T)
    
    # The vector of standard deviations for each variable; size num_factors x num_factors
    # Citation: efficient generation of sigmas and rhos from cov
    # https://github.com/aloctavodia/Statistical-Rethinking-with-Python-and-PyMC3/blob/master/Chp_13.ipynb
    sigmas = pm.Deterministic('sigmas', tt.sqrt(tt.diag(cov)))
    # correlation matrix (num_factors x num_factors)
    rhos = pm.Deterministic('rhos', tt.diag(sigmas**-1).dot(cov.dot(tt.diag(sigmas**-1))))

    # Extract the standard deviations of alpha and beta, and the correlation coefficient rho
    sigma_alpha = pm.Deterministic('sigma_alpha', sigmas[0])
    sigma_beta = pm.Deterministic('sigma_beta', sigmas[1])
    rho = pm.Deterministic('rho', rhos[0, 0])

    # Extract alpha_district and beta_district from theta_district
    alpha_district = pm.Deterministic('alpha_district', theta_district[:,0])
    beta_district = pm.Deterministic('beta_district', theta_district[:, 1])

    # Set the probability that each woman uses contraception in this model
    # It depends on the district she lives in and whether the district is urban
    # p = pm.math.invlogit(alpha + alpha_district[df.district_id] + 
    #                      (beta + beta_district[df.district_id]) * df.urban)
    p = pm.math.invlogit(alpha + theta_district[df.district_id, 0] + 
                         (beta + theta_district[df.district_id, 1]) * df.urban)

    # The response variable - whether this woman used contraception; modeled as Bernoulli
    # Bind this to the observed values
    use_contraception = pm.Bernoulli('use_contraception', p=p, observed=df['use_contraception'])

In [ ]:
# Sample from the reparameterized varying-slope model
try:
    trace_vsr = vartbl['trace_vsr']
    print(f'Loaded samples for the Variable Slopes Reparameterized model in trace_vsr.')
except:
    with model_vsr:
        nuts_kwargs = {'target_accept': 0.90}
        trace_vsr = pm.sample(draws=num_samples, tune=num_tune, nuts_kwargs=nuts_kwargs, chains=2, cores=1)
    vartbl['trace_vsr'] = trace_vsr
    save_vartbl(vartbl, fname)

# Summary of the variable-effects model
summary_vsr = pm.summary(trace_vsr)

**B4** Inspect the trace of the correlation between the intercepts and slopes, plotting the correlation marginal. What does this correlation tell you about the pattern of contraceptive use in the sample? It might help to plot the mean (or median) varying effect estimates for both the intercepts and slopes, by district. Then you can visualize the correlation and maybe more easily think through what it means to have a particular correlation. Also plot the predicted proportion of women using contraception, with urban women on one axis and rural on the other.  Finally, also plot the difference between urban and rural probabilities against rural probabilities. All of these will help you interpret your findings. (Hint: think in terms of low or high rural contraceptive use)

In [ ]:
# Run the traceplot of the correlation rho
pm.traceplot(trace_vsr, varnames=['rho'])

**Summarize Contraception Use by District for Urban vs. Rural, and Mean Parameter Values**

In [ ]:
# Add columns for urban and rural contraceptive users
df['use_contraception_urban'] = df.use_contraception * df.urban
df['use_contraception_rural'] = df.use_contraception * (1-df.urban)

# Update the aggregated contraception use in each district
agg_tbl = {
        'woman': ['count'],
        'urban': ['sum', 'mean'],
        'use_contraception': ['mean'],
        'use_contraception_urban':['sum'],
        'use_contraception_rural':['sum'],    
        }
df_district = df.groupby(by=df.district_id).agg(agg_tbl)
df_district.columns = ["_".join(x) for x in df_district.columns.ravel()]
# Compute number of rural women
df_district['rural_sum'] = df_district['woman_count'] - df_district['urban_sum']

# Change column names to make model suffix at the end of the name
df_district.rename(axis='columns', inplace=True, mapper=
    {'use_contraception_mean': 'contraception',
     'urban_sum':'urban_count',
     'rural_sum':'rural_count',
     'use_contraception_urban_sum': 'contraception_urban',
     'use_contraception_rural_sum': 'contraception_rural',
     })

# Change use_contraception_urban and use_contraception_rural to rates
df_district.contraception_urban = df_district.contraception_urban / df_district.urban_count
df_district.contraception_rural = df_district.contraception_rural / df_district.rural_count

In [ ]:
# List of parameter names for alpha_district and beta_district for each district
district_suffix = [f'district__{i}' for i in range(num_districts)]
params_alpha_district = [f'alpha_{suffix}' for suffix in district_suffix]
params_beta_district = [f'beta_{suffix}' for suffix in district_suffix]

# Get the mean of alpha_district and beta_district for all the districts; mean taken over samples
alpha_district_mean = summary_vsr.loc[params_alpha_district]['mean'].values
beta_district_mean = summary_vsr.loc[params_beta_district]['mean'].values

# Mean of "global" parameters alpha, beta, and rho
alpha_mean = summary_vsr.loc['alpha']['mean']
beta_mean = summary_vsr.loc['beta']['mean']
rho_mean = summary_vsr.loc['rho']['mean']
print(f'Mean values of the global parameters alpha, beta and rho:')
print(f'alpha: {alpha_mean:0.4f}')
print(f'beta:  {beta_mean:0.4f}')
print(f'rno:   {rho_mean:0.4f}')

# Add alpha_district, beta_district, alpha_eff, beta_eff
df_district['alpha_district'] = alpha_district_mean
df_district['beta_district'] = beta_district_mean
df_district['alpha_eff'] = alpha_mean + alpha_district_mean
df_district['beta_eff'] = beta_mean + beta_district_mean

# Compute predicted probability for urban and rural contraceptive use
df_district['pred_urban'] = logistic.cdf(df_district.alpha_eff + df_district.beta_eff)
df_district['pred_rural'] = logistic.cdf(df_district.alpha_eff)

display_cols = ['woman_count', 'urban_count', 'rural_count', 'contraception_urban', 'contraception_rural', 'contraception',
               'alpha_eff', 'beta_eff', 'pred_urban', 'pred_rural']
display(df_district[display_cols])

**Plot the mean varying effect estimates for both the incercepts and slopes, by district**

In [ ]:
# Plot beta vs. alpha
fig, ax = plt.subplots(figsize=[12,8])
ax.set_title('Mean Beta vs. Mean Alpha By District')
ax.set_xlabel('Mean alpha_district for Each District over Samples')
ax.set_ylabel('Mean beta_district for Each District over Samples')
ax.plot(alpha_district_mean, beta_district_mean, label='data', color='b', linewidth=0, marker='o', markersize=8)
ax.plot(alpha_district_mean, alpha_district_mean * rho_mean, label=r'$\rho \alpha$', linewidth=4, color='r')
ax.legend()
ax.grid()

**Plot the predicted proportion of women using contraception, with urban women on one axis and rural on the other**

In [ ]:
fig, ax1 = plt.subplots(figsize=[16,8])
ax2 = ax1.twinx()
ax1.set_title('Predicted Proportion of Women Using Contraception: Urban and Rural')
ax1.set_xlabel('District ID')
ax1.set_ylabel('Predicted % Urban Women Using Contraception')
ax2.set_ylabel('Predicted % Rural Women Using Contraception')
district_ids = df_district.index.values
h1 = ax1.plot(district_ids, df_district.pred_urban, label='urban', color='b', linewidth=0, marker='o', markersize=8)
h2 = ax2.plot(district_ids, df_district.pred_rural, label='rural', color='g', linewidth=0, marker='o', markersize=8)
ax1.legend(handles=[h1[0], h2[0]], labels=['urban', 'rural'])
ax1.grid()

In [ ]:
fig, ax = plt.subplots(figsize=[16,8])
ax.set_title('Predicted Proportion of Women Using Contraception: Urban and Rural')
ax.set_xlabel('District ID')
ax.set_ylabel('Predicted % Women Using Contraception')

ax.plot(district_ids, df_district.pred_urban, label='urban', color='b', linewidth=0, marker='o', markersize=8)
ax.plot(district_ids, df_district.pred_rural, label='rural', color='g', linewidth=0, marker='o', markersize=8)
ax.legend()
ax.grid()

**Plot the difference between urban and rural probabilities against rural probabilities**

In [ ]:
fig, ax = plt.subplots(figsize=[16,8])
ax.set_title('Predicted Difference in Urban & Rural Probs vs. Rural Prob')
ax.set_xlabel('Predicted Rural Contraceptive Prob.')
ax.set_ylabel('Predicted Urban - Rural Contraceptive Prob')
plot_x = df_district.pred_rural
plot_y = df_district.pred_urban - df_district.pred_rural
ax.plot(plot_x, plot_y, color='b', linewidth=0, marker='o', markersize=8)
ax.grid()

The first chart above is a scatter plot with data points for each of the 60 districts.  One dot represents model parameters for one district.  The x-axis is the mean value of alpha for that district (i.e. the intercept for that district).  The y-axis is the mean value of beta for that district (i.e. the change in predicted z-score for contraceptive use for urban women).

Let's put the two corners of this plot into words to develop an intuition for what's going on.  At the top left of the graph, we see one district (district 10) where alpha_district is close to -1 and beta_district is close to +1.  Let's also remember that the overall values are alpha = -0.72 and beta = 0.72.  So, in this district, alpha = -0.72 -1.0 = -1.72 and beta = 0.72 + 1.0 = +1.72.  This is saying that rural women who live in this district have a z-score for contraceptive use of -1.72, i.e. they have a 14% probability to use contraception.  The model thinks urban women have a z-score of -1.72 + 1.72 = 0.0, i.e. about half of them use contraception.  It turns out that this district has 21 women, all of whom live in rural areas and none of whom use contraception.  So the parameters describing urban women in this district are essentially noise.

At the bottom right of the chart, we see a district with an alpha_district of +1.20 and a beta_district of -1.42 (district id 33).  Adding these district level offsets to the global means, alpha in this district is -0.72 + 1.20 = 0.48 and beta is +0.72 - 1.42 = -0.70.  In this district, rural women use contraceptives with a 62% probatility while urban women use contraceptives with a 45% probability.

The next charts allow us to see that 

* Overall, urban contraception use is higher than rural contraception use
* Rural contraceptive use is quite a bit more variable than urban contraception use, which is clustered more tightly
* Therefore, in a district that has high overall contraceptive use (large alpha_district), the urban contraceptive use needs to be dialed back with a lower setting on beta_district

This is exactly the pattern that we see in the first scatter plot.

**B5** Add additional "slope" terms (one-by-one) into the model for 

- (a) the centered-age of the women and 
- (b) an indicator for whether the women have a small number or large number of existing kids in the house (you can treat 1-2 kids as low, 3-4 as high, but you might want to experiment with this split). 

In [ ]:
# Model with slope for the age of the woman;
# model_DUA stands for District, Urban, Age
with pm.Model() as model_DUA:
    # Set the prior for the overall intercept
    alpha = pm.Normal(name='alpha', mu=0.0, sd=10.0)
    # Set the prior for the overall intercept on urban, beta_age
    beta_urban = pm.Normal(name='beta_urban', mu=0.0, sd=10.0)

    # Set the prior for the intercept on age, beta_age; this is fixed only, no by district version
    # Set the sd of beta_age to 1.0 not 10.0, because the standard deviation of age_centered is 9.0
    beta_age = pm.Normal(name='beta_age', mu=0.0, sd=1.0)    
    
    # Sample the variances of the single factors by district
    sd_dist = pm.Lognormal.dist(mu=0.0, tau=1.0, shape=num_factors)
    # The parameter nu is the prior on correlation; 0 is uniform, infinity is no corelation
    eta = pm.Uniform('nu', 1.0, 5.0)
    # The number of dimensions here is 2: correlation structure is bewteen alpha and beta by district
    num_factors: int = 2
    # Sample the correlation coefficients using the LKJ distribution
    chol_packed = pm.LKJCholeskyCov('chol_packed', n=num_factors, eta=eta, sd_dist = sd_dist)
    # Expand the packed Cholesky matrix to full size
    chol = pm.Deterministic('chol', pm.expand_packed_triangular(num_factors, chol_packed))
    # Make the covariance matrix by multiplying out the cholesky factor by its transpose
    cov = pm.Deterministic('cov', tt.dot(chol, chol.T))
    # The multivariate Gaussian of (alpha, beta) by district
    # Decompose this into a "raw" part and then scale it
    theta_raw = pm.Normal(name='theta_raw', mu=0.0, sd=1.0, shape=(num_districts, num_factors))   
    # Now scale these to have the desired covariance structure
    theta_district = pm.Deterministic(name='theta_district', var=tt.dot(chol, theta_raw.T).T)
    
    # Set the probability that each woman uses contraception in this model
    # It depends on the district she lives in and whether the district is urban
    p = pm.math.invlogit(alpha + theta_district[df.district_id, 0] + 
                         (beta_urban + theta_district[df.district_id, 1]) * df.urban + 
                         beta_age * df.age_centered)

    # The response variable - whether this woman used contraception; modeled as Bernoulli
    # Bind this to the observed values
    use_contraception = pm.Bernoulli('use_contraception', p=p, observed=df['use_contraception'])

**B6** Use WAIC to compare your models. What are your conclusions?

## Q2: Mixture of experts and mixture density networks to solve inverse problems

What if you had to predict a one-to-many function? The data provided below comes from a dataset generated by Chris Bishop (yes that Bishop) to explain the models mentioned in the title above. We have included pdfs from his book which describe these models in some detail. We saw this model earlier in HW where we did an EM like algorithm to obtain a mixture of regressions.

The data is in `one-to-many.csv`

When we plot the data it looks like this. Notice both the uneven sampling (more towards the center), and the "more than one y" for a given x.

![](images/inverse.png)

Normal regression approaches to modeling such a function wont work, as they expect the function to be a proper mathematical function, that is, single valued.

These kind of problems are called **inverse problems**, where more than one input state leads to an output state, and we have to try and model these multiple input states.

A mixture of gaussians (or other distributions) might is a sensible way to do this.

You choose one of the gaussians with some probability. The nean of the gaussian is then given by some regression function, say for example a straight line. We could additionally fix the standard deviation or model it as well. 

Thus, for each component Gaussian, we choose a functional form for the mean and standard deviation. So our model looks something like this:

$$f(x)  = \sum_i \lambda_i g_i (x) $$

Say we fit a model with 3 gaussians to this data. Such a model cannot fit the function above. Notice for example that at $x=0.2$ only one of the gaussians will dominate, different from the situation at $x=0.5$. This means that the probabilities of "belonging" to one or the other gaussians is also changing with $x$.

If we allow the mixing probabilities to depend on $x$, we can model this situation.

$$f(x)  = \sum_i \lambda_i (x) g_i (x) $$

Such a model is called a "mixture of experts" model. The idea is that one "expert" gaussian is responsible in one sector of the feature space, while another expert is responsible in another sector.

You can think of this model as implementing a "standard" gaussian mixture at each "point" x, with the added complexity that all of the means, standard deviations, and mixture probabilities change from one x to another.

See https://www.cs.toronto.edu/~hinton/absps/hme.pdf and http://www.ee.hacettepe.edu.tr/~eyuksel/Publications/2012_TwentyYearsofMixtureofExperts.pdf for more details. I found the latter clearer and easier to understand.

For this entire question you might find diagram code from [here](https://github.com/hardmaru/pytorch_notebooks/blob/master/mixture_density_networks.ipynb) useful. Take with attribution.

We will assume we have **3 gaussians**.

## Part A: Variational Mixture of experts

We'll construct a gaussian mixture model of 3 "expert" linear regressions. The idea is to create a fit which looks like this:

![](images/mixreg.png)

Here the three regression lines work in different regions of $f$. We want a pricipled way to sample from this model and to be able to produce posteriors and posterior-predictives.

There are 3 parts to this model. First the means of the gaussians in the mixture are modeled with linear regression as shown in the picture above. We will also model $log(\sigma)$ for each gaussian in the mixture as a linear regression as well ($\sigma$ needs to be positive).

We now need to model the mixture probabilities, i.e., the probabilities required to choose one or the other gaussian. These mixing probabilities, the $\lambda$s will be modeled as a softmax regression (ie do a linear regression and softmax it to get 3 probabilities).

**A1** Write a pymc3 model for this problem. For all biases and weights in your regressions, assume N(0,5) priors. Add noise 0.01 to each of the three $\sigma$s to make sure you dont have a collapsed 0 width gaussian, ie we want some data in every cluster. (Thus to get the final $\sigma$, you will exponentiate your regression for $log(\sigma)$ and add 0.01.)

**A2** Fit this model variationally for about 50,000 iterations using the adam optimizer. (`obj_optimizer=pm.adam()`) Plot the ELBO to make sure you have converged. Print summaries and traceplots for the means, $\sigma$s and probabilities.

**A3** Plot the mean posteriors with standard deviations against x. Also produce a diagram like the one above to show the mean"s with standard deviations showing their uncertainty overlaid on the data.

**A4** Plot the posterior predictive (mean and variance) as a function of x) for this model (using `sample_ppc` for example). Why does the posterior predictive look nothing like the data?

**A5** Make a "correct" posterior predictive diagram by taking into account which "cluster" or "regression line" the data is coming from. To do this you will need to sample using the softmax probabilities. A nice way to do this is "Gumbel softmax sampling". See http://timvieira.github.io/blog/post/2014/07/31/gumbel-max-trick/ for details. Color-code the predictive samples with the gaussian they came from. Superimpose the predictive on the original data. You may want to contrast a prediction from a point estimate at the mean values of the $\mu$ and $\sigma$ traces at a given x (given the picked gaussian) to the "full" posterior predictive obtained from sampling from the entire trace of $\mu$ and $\sigma$ and $\lambda$. The former diagram may look something like this:

![](images/mixpred.png)

## Part B. Mixture Density Network

A mixture density network (see the enclosed Chapter 5 excerpt from Bishop or https://publications.aston.ac.uk/373/1/NCRG_94_004.pdf) is very closely related to the mixture of experts model. The difference is that we fit the regressions using a neural network where hidden layers are shared amongst the mean, sigma, and mixing probability regressions. (We could have fit 3 separate neural networks in Part A but opted to fit linear regressions for simplicity)

(More explanation [here](https://github.com/hardmaru/pytorch_notebooks/blob/master/mixture_density_networks.ipynb). You are welcome to take code from here with attribution.)

You job here is to construct a multi-layer perceptron model with a linear hidden layer with 20 units followed by a `Tanh` activation. After the activation layer, 3 separate linear layers with `n_hidden` inputs and `n_gaussian=3` outputs will complete the network. The probabilities part of the network is then passed through a softmax. The means part is left as is. The sigma part is exponentiated and 0.01 added, as in part A

Thus the structure looks like:

```
input:1 --linear-> n_hidden -> Tanh --linear-->n_gaussians      ...mu
                            --linear-->n_gaussians->softmax     ...lambda
                            --linear-->n_gaussians->exp + 0.01  ...sigma
```

We then need to use a loss function for the last layer of the network. 

Using the mean-squared-error loss is not appropriate as the expected value of samples drawn from the sampling distribution of the network will not reflect the 3-gaussian structure (this is the essence of the difference between A4 and A5 above). Thus we'll use the negative loss likelihood of the gaussian mixture model explicitly.

**B1**: Write the network as a class `MixtureDensityNetwork` which inherits from pytorch `nn.Module`. Implement a constructor which allows at-least the number of hidden layers to be varied. Also implement the `forward` method.

**B2**: Train the network using the Adam or similiar optimizer and gradient descent/SGD. Make sure your loss converges and plot this convergence.

**B3**: Plot the MLE parameters against x. Make a plot similar to A3 above where you overlay the "means" of the gaussians against the data.  Plot traces of the mu/sigma/lambda as an aid in debugging.

**B4**: Sample from the sampling distributions at the estimated point values of $\mu$ and $\sigma$ (given cluster) to make a plot similar to A5 above

**To think but not to hand in** What are the differences between a mixture density network and the mixture of experts. How do these differences translate to feature space? What would happen if we took the shared hidden layer nonlinearity (Tanh) out?

## Part C Variational Mixture Density Network

We want to implement the Mixture Density Metwork model that we constructed in Part B  directly in pymc3 and use variational inference to sample from it. We  may need more iterations in order to get convergence as this model will likely not converge as fast as the pytorch equivalent.

**C1**: Write out the equivalent pymc3 version of the MDN and generate posterior samples with ADVI.

**C2**: Sample from the posterior predictive and produce a diagram like B4 and A5 for this model. Plot traces of the mu/sigma/lambda as an aid in debugging your sampler.

**C3**: Plot the "mean" regression curves (similar to B3 and A3). Do the "mean" regression curves in this model look the same from those in Part B?  If they differ why so?

## Q3:  Exploring Temperature in Sampling and Optimiztion

At various times in class we've discussed in very vague terms the relation between "temperature" and sampling from or finding optima of distributions.  Promises would invariably be made that at some later point we'd discuss the concept of temperature and sampling/optima finding in more detail.  Let's take this problem as an opportunity to keep our promise.

Let's start by considering the function $f(x, y)$ defined in the following code cell. $f(x, y)$ is a mixture of three well separated Gaussian probability densities.

In [ ]:
make_cov = lambda  theta: np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])

theta_vec = (5.847707364986893, 5.696776968254305, 1.908095937315489)
theta1, theta2, theta3 = theta_vec

# define gaussian mixture 1 
cov1 = make_cov(theta1)
sigma1 = np.array([[2, 0],[0, 1]])
mvn1 = scipy.stats.multivariate_normal([12, 7], cov=cov1@sigma1@cov1.T)

# define gaussian mixture 2
cov2 = make_cov(theta2)
sigma2 = np.array([[1, 0],[0, 3]])
mvn2 = scipy.stats.multivariate_normal([-1, 6], cov=cov2@sigma2@cov2.T)

cov3 = make_cov(theta3)
sigma3 = np.array([[.4, 0],[0, 1.3]])
mvn3 = scipy.stats.multivariate_normal([3,-2], cov=cov3@sigma3@cov3.T)

f = lambda xvec: mvn1.pdf(xvec) + mvn2.pdf(xvec) + .5*mvn3.pdf(xvec)

p = lambda x, y: f([x,y])

## Part A Visualization and Metropolis

**A1**. Visualize $p(x, y)$ with a contour or surface plot.  Make sure to title your plot and label all axes.  What do you notice about $p(x, y)$?  Do you think it will be an easy function to sample?

**A2**. Generate 20000 samples from $p(x, y)$ using the Metropolis algorithm.  Pick individual gaussian proposals in $x$ and $y$ with $\sigma=1$, initial values, burnin parameters, and thinning parameter.  Plot traceplots of the $x$ and $y$ marginals as well as autocorrelation plots.  Plot a pathplot of your samples.  Based on your visualizations, has your Metropolis sampler generated an appropriate representation of the distribution $p(x, y)$?

A pathplot is just your samples trace overlaid on your pdf, so that you can see how the sampler traversed. It looks something like this:

![](images/pathplot.png)

## Part B: Changing pdfs using temperature

Given a function $p(x)$ we can rewrite that function in following way:

$$p(x) = e^{-(-\log(p(x))}$$

So if define the energy density for a function as $E(x)\equiv-\log{p(x)}$

We can now aim to sample from the function parameratized by a Temperature $T$.

$$p(x\vert T) = e^{-\frac{1}{T} E(x)} = p(x)^{\frac{1}{T}}$$

If we set T=1 we're sampling from our original function $p(x)$. 

**B1** In line with A1, visualize modified pdfs (dont worry about normalization) by setting the temperatures to $T=10$ and $T=0.1$. 

**B2**. Modify your Metropolis algorithm above to take a temperature parameter `T` as well as to keep track of the number of rejected proposals.  Generate 20000 samples from $p(x, y)$ at for each of the following temperatures: {0.1, 1, 3, 7, 10}. Construct  histograms of the marginals, traceplots, autocorrelation plots, and a pathplot for your samples at each temperature.  What happens to the number of rejections as temperature increases? In the limits $T \rightarrow 0$ and $T \rightarrow \infty$ what do you think your samplers will do?

**B3**. Approximate the $f(X)$ by the appropriate mixture of Gaussians as a way of generating samples from $f(X)$ to compare with other sampling methods.  Use scipy.stats.multivariate_normal to generate 20000 samples.  How do the histograms compare with the histograms for the samples from $f(X)$ at each temperature.  At what temperature do the samples best represent the function? 

## Part C: Parallel Tempering

Now that we've seen some of the pproperties of sampling at higher temperatures, let's explore a way to incorporate the improved exploration of the *entire pdf* from sampling at higher temperatures while still getting samples that match our distribution.  We'll use a technique called *parallel tempering*.  

The general idea of parallel tempering is to simulate $N$ replicas of the original system of interest (in our case, a single Metropolis Hastings chain), each replica at a different temperature. The temperature of a Metropolis Hastings Markov Chain defines how likely it is to sample from a low-density part of the target distribution. The high temperature systems are generally able to sample large volumes of parameter space, whereas low temperature systems, while having precise sampling in a local region of parameter space, may become trapped around local energy minima/probability maxima. Parallel tempering achieves good sampling by allowing the chains at different temperatures to exchange complete configurations. Thus, the inclusion of higher temperature chains ensures that the lower temperature chains can access *all* the low-temperature regions of phase space: the higher temperatures help these chains make the jump-over.

Darren Wilkinson's blog post has a [good description](https://darrenjw.wordpress.com/2013/09/29/parallel-tempering-and-metropolis-coupled-mcmc/) of whats going on.

Here is the idea that you must implement.

There are $N$ replicas each at different temperatures $T_i$ that produce $n$ samples each before possibly swapping states.

We simplify matters by only swapping states at adjacent temperatures.  The probability of swapping any two instances of the replicas is given by

$$A = min\left(1, \frac{p_k(x_{k+1})p_{k+1}(x_k)}{p_k(x_k) p_{k+1}(x_{k+1})}\right)$$

One of the $T_i$'s in our set will always be 1 and this is the only replica that we use as output of the Parallel tempering algorithm.

An algorithm for Parallel Tempering is as follows:

1. Initialize the parameters $\{(x_{init}, y_{init})_i\}, \{T_i\}, L$ where 
    * $L$ is the number of iterations between temperature swap proposals.
    * $\{T_i\}$ is a list of temperatures.  You'll run one chain at each temperature.
    * $\{(x_{init}, y_{init})_i\}$ is a list of starting points, one for each chain 
2. For each chain (one per temperature) use the simple Metropolis code you wrote earlier. Perform $L$ transitions on each chain.
3. Set the $\{(x_{init}, y_{init})_i\}$ for the next Metropolis run on each chain to the last sample for each chain i.
4. Randomly choose 2 chains at adjacent temperatures.
    1. Use the above formula to calculate the Acceptance probability $A$.
    2. With probability $A$, swap the positions between the 2 chains (that is swap the $x$s of the two chains, and separately swap the $y$s of the chains .
5. Go back to 2 above, and start the next L-step epoch 
6. Continue until you finish $Num. Samples//L$ epochs.


**C1**. Explain why swapping states with the given acceptance probability is in keeping with detailed balance. The linked blog post might help.

**C2**. Create a parallel tempering sampler that uses 5 chains at  the temperatures {0.1, 1, 3, 7, 10} to sample from $f(x, y)$.  Choose a value of L around 10-20.  Generate 10000 samples from $f(x, y)$.  Construct  histograms of the marginals, traceplots, autocorrelation plots, and a pathplot for your samples.

**C3**. How do your samples in **C2** compare to those of the Metropolis sampler?  How do they compare to the samples generated from the Gaussian Mixture approximation of $f(x, y)$?

## Part D. Global Optima using Simulated Annealing

We have new-found intuition about how to use temperature to improve our sampling. Lets now tackle the inverse idea: what happens if you sample at a lower temperature than 1. Our visualizations from Part B should indicate to us that the distributions become extremely tightly peaked arounnd their maxima.

If we initialized a metropolis-hastings sampler around an optimum at a really low temperature, it would find us a local minimum. But if we had a higher temperature at the beginning, we can use Metropolis-Hastings sampling at high temperatures to travel around the distribution and find all the peaks (valleys).  Then we will slowly cool down the temperature (which will allow us to escape local optima at higher temperatures) and finally focus us into a particular optimum region and allow you to find the optimum. It can be shown that for ceratin *temperture schedules* this method is guaranteed to find us a global minimum in the limit of infinite iterations.

We'll use this methd to find the global minimum of our distribution. The algorithm is as follows. Now we have only one chain, but we very slowly dial down its temperature to below T=1.

1. Initialize $(x, y)_i,T, L(T)$ where $L$ is the number of iterations at a particular temperature. 
2. Perform $L$ transitions thus(we will call this an epoch):
    1. Generate a new proposed position $(x, y)_{\ast}$ using 2 independent gaussians with $\sigma=1$.
    2. If $(x, y)_{\ast}$ is accepted (according to probability $P = e^{(-\Delta E/T)}$, set $(x, y)_{i+1} = (x, y)_{\ast}$, else set $(x, y)_{i+1} = x_{i}$  
3. Update T and L 
4. Until some fixed number of epochs, or until some stop criterion is fulfilled, goto 2.

$\Delta E$ is the change in enery, or the change in the negative log of the probability function. That is, $E = -log p(x,y)$. For a given T and L, this is just Metropolis!

This algorithm is called *simulated annealing* and we'll use it to find the global maximum for $f(X)$

**D1**. Use simulated annealing with a cooling schedule of $T_{k+1}=0.98T_{k}$ and a L(T) defined initially at 100 with $L_{k+1} = 1.2 L_k$ to find the global optima for $p(x, y)$.  Plot $E(x, y)$ vs iterations.  Given how we constructed $p(x, y)$ it should be fairly straight-forward to observe the  true optima by inspection.  How does the optima found by SA compare to the true optima?